# Wrangle Data

## Project Details:

This project is from Udacity's Data Wrangling course, which is part of the Data Analyst Nanodegree program. The goal is perform data wrangling, i.e. to use Python and its libraries to gather data from various sources of different formats, assess its quality and tidiness, and then clean this data. These efforts will be documented in this Jupyter Notebook.

The steps include:

- Gathering data
- Assessing this data
- Cleaning it
- Storing, analyzing, and visualizing the wrangled data
- Reporting on:
    1. Data wrangling efforts
    2. Data analyses and visualizations

*Please note some of the descriptive text in this notebook has been taken directly from the Udacity course materials.*

## Table of Contents

- [Introduction](#intro)
- [Part I: Gathering Data](#gather)
- [Part II: Assessing Data](#assess)
- [Part III: Cleaning Data](#clean)
- [Part IV: Storing, Analyzing and Visualizing Data](#analyze)
- [Conclusion](#concl)

<a id='intro'></a>
## Introduction

We will download a twitter archive and supplement it with additional data. Accoring to the Udacity course materials:

> The dataset that we will be wrangling (and analyzing and visualizing) is the tweet archive of Twitter user @dog_rates, also known as WeRateDogs. WeRateDogs is a Twitter account that rates people's dogs with a humorous comment about the dog. These ratings almost always have a denominator of 10. The numerators, though? Almost always greater than 10. 11/10, 12/10, 13/10, etc. Why? Because "they're good dogs Brent." WeRateDogs has over 4 million followers and has received international media coverage.

> WeRateDogs downloaded their Twitter archive and sent it to Udacity via email exclusively for use in this project. This archive contains basic tweet data (tweet ID, timestamp, text, etc.) for all 5000+ of their tweets as they stood on August 1, 2017.

<a id='gather'></a>
## Part I: Gathering Data

The twitter archive contains basic data for over 5000 WeRateDogs tweets. We will gather data from two additional sources in order to make our analysis more interesting. Specifically, we will need:

1. The WeRateDogs Twitter archive (twitter_archive_enhanced.csv). This file has already been placed in the same directory as this notebook.

2. Tweet image predictions that have been created with a neural network image classifier and are accessible as part of the Udacity course materials.

3. Additional information gathered by using the Twitter API, such as like and retweet counts.

Let's start by importing some necessary Python libraries for our work.

In [1]:
#Import the necessary libraries needed for the project.

# Data analysis libraries
import pandas as pd
import numpy as np

# Visualizaton tools 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Networking libraries
import requests
import tweepy
from tweepy import OAuthHandler
from timeit import default_timer as timer

# For parsing json
import json

In [2]:
# Since our data will contain long strings, make sure we display entire strings of dataframe
pd.set_option('display.max_colwidth', -1)

Let's load the WeRateDogs twitter archive into a Pandas dataframe.

In [3]:
# Create dataframe of WeRateDogs twitter archive and view the first few records.
df_twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')
df_twitter_archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",12,10,Franklin,None,None,None,None


Next we will gather the tweet image predictions, i.e., what is the dog's breed, etc. 

> Every image in the WeRateDogs Twitter archive has been run through a neural network that can classify breeds of dogs. The results include a table full of image predictions (the top three only) alongside each tweet ID, image URL, and the image number that corresponded to the most confident prediction (numbered 1 to 4 since tweets can have up to four images). This file (image_predictions.tsv) is hosted on Udacity's servers and should be downloaded programmatically using the Requests library and the following URL: https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv

In [4]:
# Set the url to the file's location
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

# Request the url and save the response
response = requests.get(url)

# If there is not an error, write the file to our directory
if response.status_code == 200:
    with open('image_predictions.tsv', 'wb') as file:
        file.write(response.content)

In [5]:
# Load the predictions into a Pandas dataframe.
df_image_predictions =  pd.read_csv('image_predictions.tsv', sep='\t')
df_image_predictions.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


Because the archive doesn't contain important information such as retweet and favorite (like) counts, we will need to get these from Twitter's API.

> Using the tweet IDs in the WeRateDogs Twitter archive, query the Twitter API for each tweet's JSON data using Python's Tweepy library and store each tweet's entire set of JSON data in a file called tweet_json.txt. Each tweet's JSON data should be written to its own line. 

*Note: The student was unable to obtain API access from Twitter, therefore the following code from the Udacity course materials has been cut and pasted into this notebook. It has not been run, as it would produce errors without the proper keys. The resulting tweet_json.txt file has been taken from the provided course materials and uploaded into the same directory as this notebook in order to simulate the final result of running this code.*

In [ ]:
# Keys, secrets, etc. are hidden to comply with Twitter's API terms and conditions
consumer_key = 'HIDDEN'
consumer_secret = 'HIDDEN'
access_token = 'HIDDEN'
access_secret = 'HIDDEN'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

# NOTE TO REVIEWER: this student had mobile verification issues so the following
# Twitter API code was sent to this student from a Udacity instructor

# Tweet IDs for which to gather additional data via Twitter's API
tweet_ids = df_twitter_archive.tweet_id.values
len(tweet_ids)

# Query Twitter's API for JSON data for each tweet ID in the Twitter archive
count = 0
fails_dict = {}
start = timer()
# Save each tweet's returned JSON as a new line in a .txt file
with open('tweet_json.txt', 'w') as outfile:
    # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
    for tweet_id in tweet_ids:
        count += 1
        print(str(count) + ": " + str(tweet_id))
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            print("Success")
            json.dump(tweet._json, outfile)
            outfile.write('\n')
        except tweepy.TweepError as e:
            print("Fail")
            fails_dict[tweet_id] = e
            pass
end = timer()
print(end - start)
print(fails_dict)

> Then, read this .txt file line by line into a pandas DataFrame with (at minimum) tweet ID, retweet count, and favorite count.

In [6]:
# Create our empty pandas dataframe (with column names only)
df_twitter_api_data = pd.DataFrame(columns = ['tweet_id', 'retweet_count', 'favorite_count'])

with open('tweet_json.txt', 'r') as file:

    # Read first line of tweet data
    tweet = file.readline()

    # Loop through successive tweets until no more left
    while (tweet):
        
        # Load json tweet data
        tweet_data = json.loads(tweet)

        # Extract and append the tweet data to our dataframe
        df_twitter_api_data = df_twitter_api_data.append({'tweet_id': tweet_data['id'], 
                          'retweet_count': tweet_data['retweet_count'], 
                          'favorite_count': tweet_data['favorite_count']
                         }, ignore_index=True)

        # Read in next tweet before going through the loop again
        tweet = file.readline()

# Print out the first few records of our dataframe
df_twitter_api_data.head()

,tweet_id,retweet_count,favorite_count
0,892420643555336193,8853,39467
1,892177421306343426,6514,33819
2,891815181378084864,4328,25461
3,891689557279858688,8964,42908
4,891327558926688256,9774,41048


<a id='assess'></a>
## Part II: Assessing Data

Now that we have gathered our data, we must assess it visually and programmatically to identify any quality and tidiness issues. Our work must meet the following project standards:

- Assessing and cleaning the entire dataset completely would require a lot of time, and is not necessary to practice and demonstrate your skills in data wrangling. Therefore, the requirements of this project are only to assess and clean at least 8 quality issues and at least 2 tidiness issues in this dataset. 
- We only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.
- Cleaning includes merging individual pieces of data according to the rules of tidy data.
- The fact that the rating numerators are greater than the denominators does not need to be cleaned. This unique rating system is a big part of the popularity of WeRateDogs.
- We do not need to gather the tweets beyond August 1st, 2017. Note that you won't be able to gather the image predictions for these tweets since you don't have access to the algorithm used.

Let's analyze each dataframe at a time starting with the archive. Then, all documented issues found will be summarized in a issues list section below.

### (1) Twitter Archive Data

We'll start by printing our dataframe and doing a quick visual assessment.

In [7]:
df_twitter_archive

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here we have a majestic great white breaching off South Africa's coast. Absolutely h*ckin breathtaking. 13/10 (IG: tucker_marlo) #BarkWeek https://t.co/kQ04fDDRmh,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950875897856/photo/1,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Meet Jax. He enjoys ice cream so much he gets nervous around it. 13/10 help Jax enjoy more things by clicking below\n\nhttps://t.co/Zr4hWfAs1H https://t.co/tVJBRMnhxl,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,https://twitter.com/dog_rates/status/890971913173991426/photo/1",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",When you watch your owner call another dog a good boy but then they turn back to you and say you're a great boy. 13/10 https://t.co/v0nONBcwxq,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/890729181411237888/photo/1,https://twitter.com/dog_rates/status/890729181411237888/photo/1",13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Zoey. She doesn't want to be one of the scary sharks. Just wants to be a snuggly pettable boatpet. 13/10 #BarkWeek https://t.co/9TwLuAGH0b,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185150312448/photo/1,13,10,Zoey,None,None,None,None
9,89024025534

There are a number of issues that we can see right away:

1. The `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp` columns all appear to contain many null (NaN) values. If a value is present in any of these columns, it indicates that the tweet is actually a reply (the first two) or a retweet (the last three). Because one of our requirements is that "we only want original ratings (no retweets)", we should remove any rows that have non-null values in any of these columns. Aftward, we can drop these columns, as they won't contain any values and they will only clutter our dataset.

2. The `source` column appears to be more wordy than it needs to be, i.e. do we actually need the hyperlink since it appears that this column is chiefly about identifying how the tweet was sent, i.e. from an iphone, the website or via another app?

3. The `text` column is actually a concatination of the tweet text, followed by the dog's rating, then a shortened hyperlink to the actual tweet itself on twitter. The rating has been split into the `rating_numerator` and `rating_denominator` columns. The full hyperlink has been expanded in the `expanded_urls` column. We could extract just the text part to isolate it and make it tidy, but unless we need this later in our analysis, this doesn't appear necessary at this time.

4. Sometimes the `expanded_urls` column contains more than one URL and scanning through them visually, they always appear to be duplicates. After opening up these URLs in a browser, it seems like there are duplicates when there is more than 1 photo in the tweet. However, these are not the URLs for the images, they are links to the original tweet itself. Unless we are going to be doing analysis on this column, fixing this issue doesn't seem to be a priority.

5. The `rating_denominator` column has at least one value that is not 10. Recall that ratings are supposed to be some number "out of 10."

6. The `name` column doesn't always contain a name. For example, there are words like "a" and "such" indicating that the extraction of the dog's name from the tweet text was not perfect. Also, there are "None" values.

7. The `doggo`, `floofer`, `pupper`, and `puppo` columns contain many "None" values. If the value is not "None", then it is simply a repeat of the column name. Also, it appears that either all of these columns within a given row are "None," or if not, then there is at most one of the four that is not "None." This is very untidy. These columns represent WeRateDog's unique  "dog stages" (i.e. what stage of life a dog is in). Since these columns all ultimately represent one variable, they should be merged into one `dog_stage` column.


Next, let's explore some of the columns programatically to better understand the issues at hand.

Let's inspect `source` column's values to determine how we should handle this column.

In [8]:
df_twitter_archive['source'].value_counts()

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                        91  
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                     33  
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>    11  
Name: source, dtype: int64

The `source` column has only a few values and they appear to represent the name of the app that was used to create the tweet, i.e. Twitter for iPhone, Vine, the Twitter Web Client (i.e. twitter.com in a browser), and TweetDeck. We clean it up by replacing the link with the actual app name.

Next, let's see if we can better understand the problem in the `name` column and understand why names were extracted incorrectly.

In [9]:
df_twitter_archive['name'].value_counts()

None         745
a            55 
Charlie      12 
Cooper       11 
Oliver       11 
Lucy         11 
Lola         10 
Tucker       10 
Penny        10 
Winston      9  
Bo           9  
Sadie        8  
the          8  
Bailey       7  
Buddy        7  
an           7  
Toby         7  
Daisy        7  
Scout        6  
Jack         6  
Koda         6  
Leo          6  
Dave         6  
Milo         6  
Rusty        6  
Bella        6  
Jax          6  
Oscar        6  
Stanley      6  
very         5  
            ..  
Alejandro    1  
Mason        1  
Miley        1  
Duddles      1  
Godi         1  
Snoop        1  
Josep        1  
by           1  
Terrenth     1  
Ozzie        1  
Einstein     1  
Lenox        1  
Philippe     1  
Jarod        1  
Lipton       1  
Laika        1  
Lacy         1  
Grey         1  
Karl         1  
Sparky       1  
Boots        1  
Mya          1  
Charl        1  
Dug          1  
space        1  
Rizzo        1  
Wafer        1  
Fynn         1

So, it appears that in addition to "None," very common English words like "a", "the", "an", "very", etc. were extracted.

Let's look at this a little further by checking the values of all names that start with a lower case letter.

In [10]:
df_twitter_archive[df_twitter_archive['name'].str.get(0).str.islower()]['name'].value_counts()

a               55
the             8 
an              7 
very            5 
one             4 
just            4 
quite           4 
getting         2 
actually        2 
mad             2 
not             2 
such            1 
by              1 
officially      1 
all             1 
old             1 
unacceptable    1 
life            1 
my              1 
incredibly      1 
this            1 
space           1 
infuriating     1 
light           1 
his             1 
Name: name, dtype: int64

There's actually not many of these, so we could replace these with nulls. Did any of the more popular of these ("a", "the" or "an") appear at the beginning of a sentence, i.e. where they would presumably be capitalized.

In [11]:
df_twitter_archive[(df_twitter_archive['name'] == 'A') | (df_twitter_archive['name'] == 'The') 
                   | (df_twitter_archive['name'] == 'An')]['name'].count()

0

These non-names appear to have been extracted mid-sentence. Let's do a quick check on capitalized names to see if we see any other instances of issues that stand out.

In [12]:
df_twitter_archive[df_twitter_archive['name'].str.get(0).str.isupper()]['name'].value_counts()

None         745
Charlie      12 
Oliver       11 
Lucy         11 
Cooper       11 
Penny        10 
Tucker       10 
Lola         10 
Bo           9  
Winston      9  
Sadie        8  
Buddy        7  
Daisy        7  
Toby         7  
Bailey       7  
Dave         6  
Milo         6  
Rusty        6  
Scout        6  
Leo          6  
Jack         6  
Bella        6  
Koda         6  
Jax          6  
Oscar        6  
Stanley      6  
Chester      5  
Sammy        5  
Phil         5  
Alfie        5  
            ..  
Poppy        1  
Richie       1  
Angel        1  
Alejandro    1  
Mason        1  
Miley        1  
Sparky       1  
Evy          1  
Fynn         1  
Wafer        1  
Strudel      1  
Tyrus        1  
Nigel        1  
Ozzie        1  
Einstein     1  
Lenox        1  
Philippe     1  
Jarod        1  
Lipton       1  
Laika        1  
Lacy         1  
Grey         1  
Terrenth     1  
Karl         1  
Boots        1  
Mya          1  
Charl        1  
Dug          1

It looks like we can concentrate our efforts on fixing the list of uncapitalized words and "None" to clean this column.

Next, let's look at the `rating_denominator` column. As we saw above, dog ratings are generally on scale of some number "/10". Let's examine this more closely to see if we can determine how the ratings were extracted incorrectly from the `text` column.

In [13]:
df_twitter_archive[df_twitter_archive['rating_denominator'] != 10][['text', 'rating_numerator', 'rating_denominator']]

,text,rating_numerator,rating_denominator
313,"@jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho",960,0
342,@docmisterio account started on 11/15/15,11,15
433,The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd,84,70
516,Meet Sam. She smiles 24/7 &amp; secretly aspires to be a reindeer. \nKeep Sam smiling by clicking and sharing this link:\nhttps://t.co/98tB8y7y7t https://t.co/LouL5vdvxx,24,7
784,"RT @dog_rates: After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https:/…",9,11
902,Why does this never happen at my front door... 165/150 https://t.co/HmwrdfEfUE,165,150
1068,"After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https://t.co/XAVDNDaVgQ",9,11
1120,Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv,204,170
1165,Happy 4/20 from the squad! 13/10 for all https://t.co/eV1diwds8a,4,20
1202,This is Bluebert. He just saw that both #FinalFur match ups are split 50/50. Amazed af. 11/10 https://t.co/Kky1DPG4iq,50,50


It appears that we run into problems when certain strings that look like ratings (i.e. "960/00") appear before the actual rating:  For example, when a date occurs before the rating (i.e. "4/20" for April 20) or when a percentage is given (i.e. 3 1/2). Sometimes, however, a rating is given for a group of dogs (i.e. 44/40) and in these cases it seems OK that we don't have the standard format of "/10."

Scanning through the list it appears there are less than 10 that are actually wrong and they can be fixed manually.

Next, let's examine the four columns that represent dog stages a little more closely (i.e. `doggo`, `floofer`, `pupper`, and `puppo`). From our initial visual assessment, it appeared that all four columns within a given row are "None" or at most one of them is has a different value. But is this actually true?

In [14]:
# Let's create a temporary dataframe that contains only rows that have at least one non-"None" value
check_dup_stages = df_twitter_archive[(df_twitter_archive['doggo']!='None') |
                  (df_twitter_archive['floofer']!='None') |
                  (df_twitter_archive['pupper']!='None') |
                   (df_twitter_archive['puppo']!='None')][['tweet_id', 'doggo', 'floofer', 'pupper', 'puppo']]

In [15]:
# Next, in order to be able to sum the number of non-None values per row, let's map
# them to numeric values. 0 for "None" and 1 for each of the column names
check_dup_stages.doggo = check_dup_stages.doggo.map({'None':0, 'doggo':1})
check_dup_stages.floofer = check_dup_stages.floofer.map({'None':0, 'floofer':1})
check_dup_stages.pupper = check_dup_stages.pupper.map({'None':0, 'pupper':1})
check_dup_stages.puppo = check_dup_stages.puppo.map({'None':0, 'puppo':1})

In [16]:
# Let's sum up our rows
check_dup_stages['stages_sum'] = check_dup_stages.doggo + check_dup_stages.floofer \
                                + check_dup_stages.pupper + check_dup_stages.puppo

In [17]:
check_dup_stages['stages_sum'].value_counts()

1    366
2    14 
Name: stages_sum, dtype: int64

It does appear that we have some rows that contain more than one dog stage. Given that there are at most 14 of them (depending on whether or not they are a reply, retween etc.), it seems that for the purpose of this analysis we can simply pick one of them when we merge these columns into a single column. Let's just verify that the only values that each of the 4 columns contains is None and a repeat of the name of that column itself.

In [18]:
df_twitter_archive['doggo'].value_counts()

None     2259
doggo    97  
Name: doggo, dtype: int64

In [19]:
df_twitter_archive['floofer'].value_counts()

None       2346
floofer    10  
Name: floofer, dtype: int64

In [20]:
df_twitter_archive['pupper'].value_counts()

None      2099
pupper    257 
Name: pupper, dtype: int64

In [21]:
df_twitter_archive['puppo'].value_counts()

None     2326
puppo    30  
Name: puppo, dtype: int64

Now that we've examined the columns that we know have issues, let's look examine our dataframe structure as a whole so we can investigate issues such as null counts and data types a little more closely.

In [22]:
df_twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

The timestamp is being stored as a string. This should be changed to datetime type.

Besides the columns for replies, retweets and stages mentioned above. It looks like the `expanded_urls` column also contains null values. Let's examine when this occurs.

In [23]:
df_twitter_archive[df_twitter_archive['expanded_urls'].isna()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
30,886267009285017600,8.862664e+17,2.281182e+09,2017-07-15 16:51:35 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",@NonWhiteHat @MayhewMayhem omg hello tanner you are a scary good boy 12/10 would pet with extreme caution,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
55,881633300179243008,8.816070e+17,4.738443e+07,2017-07-02 21:58:53 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",@roushfenway These are good dogs but 17/10 is an emotional impulse rating. More like 13/10s,NaN,NaN,NaN,NaN,17,10,None,None,None,None,None
64,879674319642796034,8.795538e+17,3.105441e+09,2017-06-27 12:14:36 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",@RealKentMurphy 14/10 confirmed,NaN,NaN,NaN,NaN,14,10,None,None,None,None,None
113,870726314365509632,8.707262e+17,1.648776e+07,2017-06-02 19:38:25 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",@ComplicitOwl @ShopWeRateDogs &gt;10/10 is reserved for dogs,NaN,NaN,NaN,NaN,10,10,None,None,None,None,None
148,863427515083354112,8.634256e+17,7.759620e+07,2017-05-13 16:15:35 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","@Jack_Septic_Eye I'd need a few more pics to polish a full analysis, but based on the good boy content above I'm leaning towards 12/10",NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
179,857214891891077121,8.571567e+17,1.806710e+08,2017-04-26 12:48:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",@Marc_IRL pixelated af 12/10,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
185,856330835276025856,NaN,NaN,2017-04-24 02:15:55 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",RT @Jenna_Marbles: @dog_rates Thanks for rating my cermets 14/10 wow I'm so proud I watered them so much,8.563302e+17,66699013.0,2017-04-24 02:13:14 +0000,NaN,14,10,None,None,None,None,None
186,856288084350160898,8.562860e+17,2.792810e+08,2017-04-23 23:26:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",@xianmcguire @Jenna_Marbles Kardashians wouldn't be famous if as a society we didn't place enormous value on what they do. The dogs are very deserving of their 14/10,NaN,NaN,NaN,NaN,14,10,None,None,None,None,None
188,855862651834028034,8.558616e+17,1.943518e+08,2017-04-22 19:15:32 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",@dhmontgomery We also gave snoop dogg a 420/10 but I think that predated your research,NaN,NaN,NaN,NaN,420,10,None,None,None,None,None
189,855860136149123072,8.558585e+17,1.361572e+07,2017-04-22 19:05:32 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","@s8n You tried very hard to portray this good boy as not so good, but you have ultimately failed. His goodness shines through. 666/10",NaN,NaN,NaN,NaN,666,10,None,None,None,None,None


All but four of these instances of null values are actually replies and one of those four is actually a retweet. Therefore, most of these rows will be removed from our set anyway. Let's look at the remaining three:

In [24]:
df_twitter_archive[(df_twitter_archive['tweet_id'] == 828361771580813312) 
                   | (df_twitter_archive['tweet_id'] == 785515384317313025) 
                   | (df_twitter_archive['tweet_id'] == 696518437233913856)]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
375,828361771580813312,NaN,NaN,2017-02-05 21:56:51 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Twitter Web Client</a>",Beebop and Doobert should start a band 12/10 would listen,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
707,785515384317313025,NaN,NaN,2016-10-10 16:20:36 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","Today, 10/10, should be National Dog Rates Day",NaN,NaN,NaN,NaN,10,10,None,None,None,None,None
1445,696518437233913856,NaN,NaN,2016-02-08 02:18:30 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Oh my god 10/10 for every little hot dog pupper,NaN,NaN,NaN,NaN,10,10,None,None,None,pupper,None


The issue appears to be that these three are missing the URL at the end of the text field, hence it can't be extracted into the `expanded_urls column`. If we plug the tweet id's into the twitter url format (for example, https://twitter.com/dog_rates/status/696518437233913856) and visit each of them in a browser, we can see that these tweets don't have photos. In order to follow our project's requirements, we will need to exclude these three rows.

Finally, let's verify that our dataset doesn't contain duplicates.

In [25]:
print('Duplicates: {}'.format(sum(df_twitter_archive.duplicated())))

Duplicates: 0


That wraps up our initial assement of the archive data!

### (2) Image Predictions

As per the Udacity project description, the image predictions file is:
> a table full of image predictions (the top three only) alongside each tweet ID, image URL, and the image number that corresponded to the most confident prediction (numbered 1 to 4 since tweets can have up to four images).
> So for the last row in that table:
> - tweet_id is the last part of the tweet URL after "status/" → https://twitter.com/dog_rates/status/889531135344209921
- p1 is the algorithm's #1 prediction for the image in the tweet → golden retriever
- p1_conf is how confident the algorithm is in its #1 prediction → 95%
- p1_dog is whether or not the #1 prediction is a breed of dog → TRUE
- p2 is the algorithm's second most likely prediction → Labrador retriever
- p2_conf is how confident the algorithm is in its #2 prediction → 1%
- p2_dog is whether or not the #2 prediction is a breed of dog → TRUE
- etc.

Again, we'll start our assesment by printing our dataframe and doing a quick visual assessment.

In [26]:
df_image_predictions

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True


The first thing that jumps out is that the values `p1`, `p2` and `p3` columns are dirty. There is inconsistent capitalization and underscores between words. We will need to clean these columns.

At a first glance all values appear to be present. However, we'll need to verify programatically later if nulls are present.

Another thing that stands out is `jpg_url` column. This contains a link to each dog photo in Twitter's image hosting domain. Since tweets can contain up to four images, we should dertermine if tweet id's are repeated:

In [27]:
sum(df_image_predictions['tweet_id'].duplicated())

0

There are no duplicate tweet id's, which will make it easier to merge this dataset with the archive, if necessary.

Next, let's confirm that there are no nulls and verify that our data types are correct.

In [28]:
df_image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


Our dataset contains no nulls and the data types for each column appear to be correct.

One thing that stands out, however, is that we have substantially fewer rows in this dataset than in the archive. Let's compare actual counts.

In [29]:
print('Rows in df_twitter_archive: {}'.format(df_twitter_archive.shape[0]))
print('Rows in df_image_predictions: {}'.format(df_image_predictions.shape[0]))

Rows in df_twitter_archive: 2356
Rows in df_image_predictions: 2075


Recall that we will need to remove some rows from the archive because they are replies or retweets. There were also 3 regular tweets that do not contain images that we will need to remove. Let's get a total count of the archive rows that will remain and compare it to our image prediction row size.

In [30]:
print('Replies to remove: {}'.format(df_twitter_archive[~df_twitter_archive['in_reply_to_status_id'].isna()].tweet_id.count()))
print('Retweets to remove: {}'.format(df_twitter_archive[~df_twitter_archive['retweeted_status_id'].isna()].tweet_id.count()))

Replies to remove: 78
Retweets to remove: 181


So, we already know we will be exlcuding 262 records (3 + 78 + 181), which leaves us with a total of about 2094 df_twitter_archive rows, which is still higher than the 2075 image predictions.

Bottom line: There are about 19 tweets in the archive for which we will not have image predictions.

This brings us to our last point. From a tidiness standpoint, while we there is a case to be made to store this information in it's own separate dataframe/table, it will facilitate our analysis if it is moved into the same dataframe as our archive data.

### (3) API data

In [31]:
df_twitter_api_data

,tweet_id,retweet_count,favorite_count
0,892420643555336193,8853,39467
1,892177421306343426,6514,33819
2,891815181378084864,4328,25461
3,891689557279858688,8964,42908
4,891327558926688256,9774,41048
5,891087950875897856,3261,20562
6,890971913173991426,2158,12041
7,890729181411237888,16716,56848
8,890609185150312448,4429,28226
9,890240255349198849,7711,32467


There are no glaring issues that are visible when scrolling through the dataset. Let's check for nulls and mismatched datatypes programatically.

In [32]:
df_twitter_api_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 3 columns):
tweet_id          2354 non-null object
retweet_count     2354 non-null object
favorite_count    2354 non-null object
dtypes: object(3)
memory usage: 55.2+ KB


There are no nulls, however all of our columns are numbers (specifically, integers), yet they are being stored as text. Retweet_count and favorite_count should be changed to integer types, as we will want to be able to perform numeric and statistical funtions on them. Initially, it might not seem necessary to convert the tweet_id column, as we will not be performing mathematical functions on it, however if we wish to join our data with the archive data (where tweet_id is an integer), our column should be of the same type.

Next let's check to see if there are any duplicates.

In [33]:
print('Duplicates: {}'.format(sum(df_twitter_api_data.duplicated())))

Duplicates: 0


Note, however, that there are only 2354 rows in the api data, but there are 2356 rows in our archive. While we will be removing replies, retweets and tweets without photos, it is possible that there could be some missing data.

Finally, from a tidiness perspective, there is no reason this data needs to be in its own separate dataframe. In fact, it should be merged into the larger twitter archive dataframe as retweet and favorite counts are essential to a tweet.

### Assesment Summary: Issues List

Below is a list of our identified quality and tidiness issues for all dataframes that we will need to clean.

#### Quality

##### `df_twitter_achive` dataframe
- Replies and retweets are included (i.e. there are non-nulls in the `in_reply_to_status_id`, `in_reply_to_user_id`, `in retweeted_status_id`, `retweeted_status_user_id`, and `retweeted_status_timestamp` columns.)
- Some tweets have no photo (i.e. the `expanded_urls` column contains nulls.)
- `name` contains erroneous values ("a", "such", "None")
- The `rating_denominator` was not always extracted correctly, especially if there was a date or percentage preceding the rating in the tweet.
- `source` column contains unnecessary link information. We can strip it and just keep the actual app name.
- `timestamp` has the wrong data type (it should be datetime and not text)


##### `df_image_predictions` dataframe
- The `p1`, `p2` and `p3` columns are dirty (i.e. capitalization is inconsistent and there are underscores between words.)
- Not all tweets in the archive have an image prediction. There are 2075 image predictions and 2356 tweets in the archive. Even after we remove replies, retweets and tweets without photos there will still be 2094 tweets in the archive.

##### `df_twitter_api_data` dataframe
- `tweet_id`, `retweet_count` and `favorite_count` should be converted to integer format (to be consistent with our other dataframes.)
- Not all tweets in the archive have associated api data. There are only 2354 rows in the api data, but there are 2356 rows in our archive.

#### Tidiness

##### `df_twitter_achive` dataframe
- The `doggo`, `floofer`, `pupper`, and `puppo` columns all represent a single idea and should be converted into a single `dog_stage` column.
- After removing the non-nulls in `in_reply_to_status_id`, `in_reply_to_user_id`, `in retweeted_status_id`, `retweeted_status_user_id`, and `retweeted_status_timestamp`, we should remove these columns as they clutter our dataset.

##### `df_image_predictions` dataframe
- This dataset should be merged into the larger tweet archive, joined on `tweet_id`

##### `df_twitter_api_data` dataframe
- The `retweet_count` and `favorite_count` columns should be merged into the larger tweet archive, joined on `tweet_id` (which needs to be converted to an integer type first)



<a id='clean'></a>
## Part III: Cleaning Data

> Clean each of the issues you documented while assessing. The result should be a high quality and tidy master pandas DataFrame (or DataFrames, if appropriate).

The first thing that need to do copy our datasets to a new frames before we start cleaning them.

In [34]:
df_archive_clean = df_twitter_archive.copy()
df_image_predictions_clean = df_image_predictions.copy()
df_api_data_clean = df_twitter_api_data.copy()

Next, we'll adress the issues above in an organized fashion. We will start by handling issues with missing data. Then, we will address our tidiness issues. Finally, we will fix the quality issues.

### Missing Data

#### Replies and retweets are included in our archive dataset.

##### Define
Keep only rows that contain null values in the `in_reply_to_status_id` and `in retweeted_status_id` columns. This will "delete" all of the replies and retweets respectively.

##### Code

In [35]:
# Remove all replies
df_archive_clean = df_archive_clean[df_archive_clean['in_reply_to_status_id'].isnull()]

In [36]:
# Remove all retweets
df_archive_clean = df_archive_clean[df_archive_clean['retweeted_status_id'].isnull()]

##### Test

In [37]:
# Verify that there are no non-null values in the 'in_reply_to_status_id' or 'in_reply_to_user_id' columns
# As these are the two columns associated with replies, we should see '0' returned
df_archive_clean[(~df_archive_clean['in_reply_to_status_id'].isnull()) | (~df_archive_clean['in_reply_to_user_id'].isnull())]['tweet_id'].count()

0

In [38]:
# Verify that there are no non-null values in the in 'retweeted_status_id', 'retweeted_status_user_id', 
# or 'retweeted_status_timestamp' columns
# As these are the two columns associated with retweets, we should see '0' returned
df_archive_clean[(~df_archive_clean['retweeted_status_id'].isnull()) | (~df_archive_clean['retweeted_status_user_id'].isnull()) | (~df_archive_clean['retweeted_status_timestamp'].isnull())]['tweet_id'].count()

0

#### Tweets have no photo are included in our archive dataset.

##### Define
Remove rows that have nulls in the `expanded_urls` column.

##### Code

In [39]:
# Remove rows with null in 'expanded_urls' column
df_archive_clean = df_archive_clean[~df_archive_clean['expanded_urls'].isnull()]

##### Test

In [40]:
# Verify that the number of rows with null in 'expanded_urls' column is 0
df_archive_clean[df_archive_clean['expanded_urls'].isnull()]['tweet_id'].count()

0

In [41]:
# Let's do a quick count on the number of remaining rows in the archive. It should be 2094, as calculated in the 
# Assess section
df_archive_clean.shape[0]

2094

#### Not all tweets in the archive have an image prediction.

There are 2075 image predictions and 2356 tweets in the archive. Even after we remove replies, retweets and tweets without photos there will still be 2094 tweets in the archive. There will be at most 19 tweets that don't have associated image predictions. It seems safe to be able to exclude these from our analysis. If we perform an inner join when merging the image predictions into the archive dataset, this issue will be addressed as a result. There's no action required at this time.

#### Not all tweets in the archive have associated api data. 

There are only 2354 rows in the api data, but there are 2356 rows in our archive. There will be at most 2 tweets that don't have associated api data. It seems safe to be able to exclude these from our analysis. If we perform an inner join when merging the api data into the archive dataset, this issue will be addressed as a result. There's no action required at this time.

### Tidiness

#### The columns associated with replies and retweets should be removed from the twitter archive.

##### Define

Drop the `in_reply_to_status_id`, `in_reply_to_user_id`, `in retweeted_status_id`, `retweeted_status_user_id`, and `retweeted_status_timestamp` columns from `df_archive_clean`.

##### Code

In [42]:
df_archive_clean = df_archive_clean.drop(['in_reply_to_status_id', 
                                          'in_reply_to_user_id', 
                                          'retweeted_status_id', 
                                          'retweeted_status_user_id', 
                                          'retweeted_status_timestamp'], axis=1)

##### Test

In [43]:
# Verify that the above columns are not present in the list of returned columns
df_archive_clean.columns

Index(['tweet_id', 'timestamp', 'source', 'text', 'expanded_urls',
       'rating_numerator', 'rating_denominator', 'name', 'doggo', 'floofer',
       'pupper', 'puppo'],
      dtype='object')

#### The `doggo`, `floofer`, `pupper`, and `puppo` columns of the image predictions dataset should be converted into a single `dog_stage` column.

##### Define

Merging columns just screams for using the pandas ["melt" function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.melt.html). We pick our set of columns to be used as the "id" for each row (this will be all of the columns that we're not merging or "melting" together), as well as the four that we are melting together (these are the "values"). This will reduce our four columns down to two: one for the original column name (the "variable") and one for it its "value." Note that in our case this will create four rows for each tweet id. We don't need the "variable" column, so we can drop that. Now, if only we could we could easily pick the one of these four rows that we need to keep. Fortunately, the Panda's sort_values function can help. Becuase it will sort capitalized words before uncapitalized ones, we will easily be able to identify a non-"None" value if one exists, as it will be the last one after sorting. Then all we need to do is to drop the preceding rows. As we saw in the Assess section above, there is the rare case where there is actually more than one dog stage. In this case we will chose the one that gets sorted to the last position.

To summarize, in order to "melt" the above four columns into a single `dog_stages` column, we will have to perform the following steps:
1. Prepare the parameters we need in order to use the Panda's melt command.
2. Perform the actual melt.
3. Delete the unneeded "variable" column.
4. Sort our resulting dataframe by `dog_stage`, causing rows with a "None" stage to float to the top.
5. Drop the "duplicate" rows for each tweet_id, keeping only the one the last sorted `dog_stage` "value."

##### Code

Before we begin, lets get a count of how many of our tweets contain a value in one of the 4 columns named above. This number should match the number of records that have a value in the newly created `dog_stage` column when we are done.

In [44]:
df_archive_clean[(df_archive_clean['doggo']!='None') |
                  (df_archive_clean['floofer']!='None') |
                  (df_archive_clean['pupper']!='None') |
                   (df_archive_clean['puppo']!='None')].tweet_id.count()

335

In [45]:
# Columns used for id_vars parameter in the melt will be all but the last 4 in our archive
id_cols = df_archive_clean.columns[:-4].values.tolist()
id_cols

['tweet_id',
 'timestamp',
 'source',
 'text',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'name']

In [46]:
# Columns used in the value_vars parameter in melt will be the last 4 in our archive 
# (i.e. doggo, floofer, pupper, and puppo)
value_cols = df_archive_clean.columns[8:].values.tolist()
value_cols

['doggo', 'floofer', 'pupper', 'puppo']

In [47]:
# Perform the melt. This will create a row for each tweet for each of the columns specified in the value_vars
# The "value" of that original column will be placed in the "dog_stage" column (specified in the value_name paramenter)
df_archive_clean = pd.melt(df_archive_clean, id_vars = id_cols, value_vars = value_cols, value_name='dog_stage')

In [48]:
# We won't need the variable column, so let's drop it
df_archive_clean = df_archive_clean.drop(['variable'], axis=1)

In [49]:
# Sort on the tweet_id and dog_stage column
df_archive_clean = df_archive_clean.sort_values(by=['dog_stage'])

In [50]:
# Let's drop the unnecessary rows for each tweet id (our subset), use keep='last' to make sure
# that any non-None values are chosen.
df_archive_clean = df_archive_clean.drop_duplicates(subset='tweet_id', keep='last')

##### Test

In [51]:
# Verify that we haven't lost any rows. This should return 2094.
df_archive_clean.shape[0]

2094

In [52]:
# Verify that we have the same number of rows that contain a non-None dog_stage value that we did above,
# i.e. this should return 335.
df_archive_clean[df_archive_clean['dog_stage']!='None'].tweet_id.count()

335

#### The image predictions dataset should be merged into the larger tweet archive, joined on `tweet_id`

##### Define

- Determine the total number of rows that should result from the merge so that we can test our result
- Merge `df_image_predictions_clean` into `df_archive_clean` with a left join on `tweet_id`

##### Code

In [53]:
# Create a set of the tweet ids in our archive
archive_tweet_ids = set(df_archive_clean['tweet_id'].tolist())

# Create a set of the tweet ids in our image predictions
prediction_tweet_ids = set(df_image_predictions_clean['tweet_id'].tolist())

# Get the set of ids that are in the archive that are not in the image predictions file
missing_tweet_ids = archive_tweet_ids.difference(prediction_tweet_ids)

# Print the total number of id's that will be present in our merged sets
len(archive_tweet_ids) - len(missing_tweet_ids)

1971

In [54]:
# Merge the two dataframes
df_archive_clean = df_archive_clean.merge(df_image_predictions_clean, how='inner', on='tweet_id') 

##### Test

In [55]:
# Verify that the newly merged data is present
# Specifically that the jpg_url, img_num, p1, p1_conf, p1_dog, p2, p2_conf, p2_dog, p3, p3_conf, and p3_dog columns are present
df_archive_clean.head()

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,dog_stage,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,667435689202614272,2015-11-19 20:14:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Ermergerd 12/10 https://t.co/PQni2sjPsm,https://twitter.com/dog_rates/status/667435689202614272/photo/1,12,10,None,None,https://pbs.twimg.com/media/CUM1OHCW4AEgGSi.jpg,1,Rottweiler,0.999091,True,miniature_pinscher,0.000450,True,black-and-tan_coonhound,0.000157,True
1,667437278097252352,2015-11-19 20:20:22 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Never seen this breed before. Very pointy pup. Hurts when you cuddle. Still cute tho. 10/10 https://t.co/97HuBrVuOx,https://twitter.com/dog_rates/status/667437278097252352/photo/1,10,10,None,None,https://pbs.twimg.com/media/CUM2qWaWoAUZ06L.jpg,1,porcupine,0.989154,False,bath_towel,0.006300,False,badger,0.000966,False
2,667443425659232256,2015-11-19 20:44:47 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Exotic dog here. Long neck. Weird paws. Obsessed with bread. Waddles. Flies sometimes (wow!). Very happy dog. 6/10 https://t.co/rqO4I3nf2N,https://twitter.com/dog_rates/status/667443425659232256/photo/1,6,10,None,None,https://pbs.twimg.com/media/CUM8QZwW4AAVsBl.jpg,1,goose,0.980815,False,drake,0.006918,False,hen,0.005255,False
3,667453023279554560,2015-11-19 21:22:56 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Twitter Web Client</a>",Meet Cupcake. I would do unspeakable things for Cupcake. 11/10 https://t.co/6uLCWR9Efa,https://twitter.com/dog_rates/status/667453023279554560/photo/1,11,10,Cupcake,None,https://pbs.twimg.com/media/CUNE_OSUwAAdHhX.jpg,1,Labrador_retriever,0.825670,True,French_bulldog,0.056639,True,Staffordshire_bullterrier,0.054018,True
4,667455448082227200,2015-11-19 21:32:34 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Twitter Web Client</a>",This is Reese and Twips. Reese protects Twips. Both think they're too good for seat belts. Simply reckless. 7/10s https://t.co/uLzRi1drVK,https://twitter.com/dog_rates/status/667455448082227200/photo/1,7,10,Reese,None,https://pbs.twimg.com/media/CUNHMXTU8AAS3HH.jpg,1,Tibetan_terrier,0.676376,True,Irish_terrier,0.054933,True,Yorkshire_terrier,0.040576,True


In [56]:
# The total number of rows in the archive should now match what we calculated above
df_archive_clean.shape[0]

1971

#### The api data should be merged into the larger tweet archive, joined on `tweet_id` (which needs to be converted to an integer type first)

##### Define

- Convert the `tweet_id` column of `df_api_data_clean` to an integer
- Determine the total number of rows that should result from the merge so that we can test our result
- Merge `df_api_data_clean` into `df_archive_clean` with a left join on `tweet_id``

##### Code

In [57]:
# Convert the tweet_id column of the api data to an integer
df_api_data_clean['tweet_id'] = df_api_data_clean['tweet_id'].astype(int)

In [58]:
# Create a set of the tweet ids in our archive
archive_tweet_ids = set(df_archive_clean['tweet_id'].tolist())

# Create a set of the tweet ids in our api data
api_tweet_ids = set(df_api_data_clean['tweet_id'].tolist())

# Get the set of ids that are in the archive that are not in the image predictions file
missing_tweet_ids = archive_tweet_ids.difference(api_tweet_ids)

# Print the total number of id's that will be present in our merged sets
len(archive_tweet_ids) - len(missing_tweet_ids)

1971

In [59]:
# Merge the two dataframes
df_archive_clean = df_archive_clean.merge(df_api_data_clean, how='inner', on='tweet_id')

##### Test

In [60]:
# Verify that the newly merged data is present, specifically the retweet_count and favorite_count columns
df_archive_clean.head()

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,dog_stage,jpg_url,...,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,retweet_count,favorite_count
0,667435689202614272,2015-11-19 20:14:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Ermergerd 12/10 https://t.co/PQni2sjPsm,https://twitter.com/dog_rates/status/667435689202614272/photo/1,12,10,None,None,https://pbs.twimg.com/media/CUM1OHCW4AEgGSi.jpg,...,0.999091,True,miniature_pinscher,0.000450,True,black-and-tan_coonhound,0.000157,True,89,326
1,667437278097252352,2015-11-19 20:20:22 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Never seen this breed before. Very pointy pup. Hurts when you cuddle. Still cute tho. 10/10 https://t.co/97HuBrVuOx,https://twitter.com/dog_rates/status/667437278097252352/photo/1,10,10,None,None,https://pbs.twimg.com/media/CUM2qWaWoAUZ06L.jpg,...,0.989154,False,bath_towel,0.006300,False,badger,0.000966,False,257,483
2,667443425659232256,2015-11-19 20:44:47 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Exotic dog here. Long neck. Weird paws. Obsessed with bread. Waddles. Flies sometimes (wow!). Very happy dog. 6/10 https://t.co/rqO4I3nf2N,https://twitter.com/dog_rates/status/667443425659232256/photo/1,6,10,None,None,https://pbs.twimg.com/media/CUM8QZwW4AAVsBl.jpg,...,0.980815,False,drake,0.006918,False,hen,0.005255,False,620,833
3,667453023279554560,2015-11-19 21:22:56 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Twitter Web Client</a>",Meet Cupcake. I would do unspeakable things for Cupcake. 11/10 https://t.co/6uLCWR9Efa,https://twitter.com/dog_rates/status/667453023279554560/photo/1,11,10,Cupcake,None,https://pbs.twimg.com/media/CUNE_OSUwAAdHhX.jpg,...,0.825670,True,French_bulldog,0.056639,True,Staffordshire_bullterrier,0.054018,True,96,327
4,667455448082227200,2015-11-19 21:32:34 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Twitter Web Client</a>",This is Reese and Twips. Reese protects Twips. Both think they're too good for seat belts. Simply reckless. 7/10s https://t.co/uLzRi1drVK,https://twitter.com/dog_rates/status/667455448082227200/photo/1,7,10,Reese,None,https://pbs.twimg.com/media/CUNHMXTU8AAS3HH.jpg,...,0.676376,True,Irish_terrier,0.054933,True,Yorkshire_terrier,0.040576,True,66,203


In [61]:
# The total number of rows in the archive should be equal to the count tallied above
df_archive_clean.shape[0]

1971

### Quality

#### `timestamp` has the wrong data type (it should be datetime and not text)

##### Define

Convert the `timestamp`column to datetime type

##### Code

In [62]:
df_archive_clean['timestamp'] = pd.to_datetime(df_archive_clean['timestamp'])

##### Test

In [63]:
# Verify that timestamp has datetime data type
df_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1971 entries, 0 to 1970
Data columns (total 22 columns):
tweet_id              1971 non-null int64
timestamp             1971 non-null datetime64[ns]
source                1971 non-null object
text                  1971 non-null object
expanded_urls         1971 non-null object
rating_numerator      1971 non-null int64
rating_denominator    1971 non-null int64
name                  1971 non-null object
dog_stage             1971 non-null object
jpg_url               1971 non-null object
img_num               1971 non-null int64
p1                    1971 non-null object
p1_conf               1971 non-null float64
p1_dog                1971 non-null bool
p2                    1971 non-null object
p2_conf               1971 non-null float64
p2_dog                1971 non-null bool
p3                    1971 non-null object
p3_conf               1971 non-null float64
p3_dog                1971 non-null bool
retweet_count         1971 non-n

#### `retweet_count` and `favorite_count` should be converted to integer format 

##### Define

- Convert retweet_count to integer
- Convert favorite_count to integer

##### Code

In [64]:
df_archive_clean['retweet_count'] = df_archive_clean['retweet_count'].astype(int)
df_archive_clean['favorite_count'] = df_archive_clean['favorite_count'].astype(int)

##### Test

In [65]:
# Verify that both columns are now of data type integer
df_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1971 entries, 0 to 1970
Data columns (total 22 columns):
tweet_id              1971 non-null int64
timestamp             1971 non-null datetime64[ns]
source                1971 non-null object
text                  1971 non-null object
expanded_urls         1971 non-null object
rating_numerator      1971 non-null int64
rating_denominator    1971 non-null int64
name                  1971 non-null object
dog_stage             1971 non-null object
jpg_url               1971 non-null object
img_num               1971 non-null int64
p1                    1971 non-null object
p1_conf               1971 non-null float64
p1_dog                1971 non-null bool
p2                    1971 non-null object
p2_conf               1971 non-null float64
p2_dog                1971 non-null bool
p3                    1971 non-null object
p3_conf               1971 non-null float64
p3_dog                1971 non-null bool
retweet_count         1971 non-n

#### `name` contains erroneous values ("a", "such", "None")

##### Define

##### Code

##### Test

#### The `rating_denominator` was not always extracted correctly, especially if there was a date or percentage preceding the rating in the tweet.

##### Define

##### Code

##### Test

#### `source` column contains unnecessary link information. We can strip it and just keep the actual app name.

##### Define

##### Code

##### Test

#### The `p1`, `p2` and `p3` columns are dirty (i.e. capitalization is inconsistent and there are underscores between words.)

##### Define

##### Code

##### Test

<a id='analyze'></a>
## Part IV: Storing, Analyzing and Visualizing Data

> Store the clean DataFrame(s) in a CSV file with the main one named twitter_archive_master.csv. If additional files exist because multiple tables are required for tidiness, name these files appropriately. Additionally, you may store the cleaned data in a SQLite database (which is to be submitted as well if you do).

<a id='concl'></a>
## Conclusion

#### Reporting for this Project

> Create a 300-600 word written report called wrangle_report.pdf or wrangle_report.html that briefly describes your wrangling efforts. This is to be framed as an internal document.

> Create a 250-word-minimum written report called act_report.pdf or act_report.html that communicates the insights and displays the visualization(s) produced from your wrangled data. This is to be framed as an external document, like a blog post or magazine article, for example.

> Both of these documents can be created in separate Jupyter Notebooks using the Markdown functionality of Jupyter Notebooks, then downloading those notebooks as PDF files or HTML files (see image below). You might prefer to use a word processor like Google Docs or Microsoft Word, however.